# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


In [6]:
class Matrix:
    def __init__(self, n= None, m= None, data= None):
        """
        With arguments n and m, the size of the matrix filled with zeros.
        With a list of lists of values.
        """
        if data is not None:
            # initialize with list of lists
            self.data = data
            self.n= len(self.data)
            if self.n == 0:
                self.m = 0
            else:
                self.m = len(self.data[0])

                # check if all rows have the same number of columns
                if not all(len(row) == self.m for row in self.data):
                    raise ValueError("All rows must have the same number of columns.")
        elif n is not None and m is not None:
            if n <= 0 or m <= 0:
                raise ValueError("Dimensions n and m must be positive integers!")
            self.n = n
            self.m = m
            self.data = [[0 for _ in range (m) ] for _ in range (n)]
        else:
            raise ValueError("Either (n and m) or data must be provided for Matric initiation.")
            

    def __getitem__(self, index):
        """
        Matrix indexing: supports M[i][j] and M[i,j]
        """
        if isinstance(index, tuple):
            if len(index) != 2:
                raise IndexError("Invalid index. Use M[i][j] or M[i, j].")
            i, j = index
            return self.data[i][j]
        elif isinstance(index, int):
            return self.data[index]
        else:
            raise IndexError("Invalid index type.")
    
    def __setitem__(self, index, value):
        """
        Matrix assignment: supports M[i][j] = value and M[i,j] = value
        """
        if isinstance(index, tuple):
            if len(index) != 2:
                raise IndexError("Invalid index. Use M[i][j] or M[i, j].")
            i, j = index
            self.data[i][j] = value
        elif isinstance(index, int):
            self.data[index] = value
        else:
            raise IndexError("Invalid index type.")
    
    def __eq__(self, other):
        """
        Matrix equality check
        """
        if isinstance(other, Matrix):
            return self.n == other.n and self.m == other.m and self.data == other.data
        elif isinstance(other, list):
            return self.n == len(other) and self.m == len(other[0]) and self.data == other
        else:
            return False
    
    def assign(self, other):
        """
        Matrix assignment: set values of self to those of other
        """
        if isinstance(other, Matrix):
            if self.n != other.n or self.m != other.m:
                raise ValueError("Matrices must be the same size for assignment.")
            self.data = [row[:] for row in other.data]
        elif isinstance(other, list):
            if self.n != len(other) or self.m != len(other[0]):
                raise ValueError("Matrices must be the same size for assignment.")
            self.data = [row[:] for row in other]
        else:
            raise TypeError("Invalid type for assignment!")
    
    def __repr__(self):
        """
        String representation of the matrix for printing
        """
        return '\n'.join([' '.join(f"{val:4}" for val in row) for row in self.data])     

In [7]:
# Test 1: Initialize  a 2x3 zero matrix.
M1 = Matrix(n=2, m=3)
print("M1 (2x3 zero matrix): ")
print(M1)

M1 (2x3 zero matrix): 
   0    0    0
   0    0    0


In [8]:
# Test 2: Initialize a matrix from list of lists.
M2 = Matrix(data = [[1,2,3], [4,5,6]])
print("M2 (Initialize from list of lists): ")
print(M2)

M2 (Initialize from list of lists): 
   1    2    3
   4    5    6


In [9]:
# Test 2: Indexing
M2 = Matrix(data = [[1,2,3], [4,5,6]])
print("Indexing")
print("M2 [0] [1]:", M2[1][1]) # output 5
print("M2 [0] [1]:", M2[1][2]) # output 6

Indexing
M2 [0] [1]: 5
M2 [0] [1]: 6


In [10]:
# Test 4 Assignment
M2[0,2] = 45
print("M2 after M2[1] [2]= 45:")
print(M2)

M2 after M2[1] [2]= 45:
   1    2   45
   4    5    6


In [11]:
# Test 5: Matrix equality
M3 = Matrix(data=[[1, 2, 45], [4, 5, 6]])
print("M2 == M3:", M2 == M3)  # output True

M2 == M3: True


In [12]:
# Test 6: Matrix assignment
M4 = Matrix(n=2, m=3)
M4.assign(M2)  # Use the public method, not the dunder method
print("M4 after assignment from M2:")
print(M4)

M4 after assignment from M2:
   1    2   45
   4    5    6


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

In [13]:
class Matrix:
    def __init__(self, n= None, m= None, data= None):
        """
        With arguments n and m, the size of the matrix filled with zeros.
        With a list of lists of values.
        """
        if data is not None:
            # Initialize with a list of lists
            self.data = data
            self.n = len(self.data)
            if self.n == 0:
                self.m = 0
            else:
                self.m = len(self.data[0])
                # Check if all rows have the same number of columns
                if not all(len(row) == self.m for row in self.data):
                    raise ValueError("All rows must have the same number of columns.")
        elif n is not None and m is not None:
            # Initialize with n and m (size of the matrix)
            if n <= 0 or m <= 0:
                raise ValueError("Dimensions n and m must be positive integers!")
            self.n = n
            self.m = m
            self.data = [[0 for _ in range(m)] for _ in range(n)]
        else:
            raise TypeError("Either (n and m) or data must be provided for Matrix initialization.")
    
    def __getitem__(self, index):
        """
        Matrix indexing: supports M[i][j], M[i,j], and slicing.
        """
        if isinstance(index, tuple):
            if len(index) != 2:
                raise IndexError("Invalid index! Use M[i][j] or M[i, j].")
            i, j = index
            # handle slicing
            if isinstance(i, slice) or isinstance(j, slice):
                # get sliced rows
                rows = self.data[i]
                # get sliced columns from each row
                sliced_data = [row[j] for row in rows]
                return Matrix(data=sliced_data)
            else:
                return self.data[i][j]
        elif isinstance(index, int):
            return self.data[index]
        else:
            raise IndexError("Invalid index type!")
    
    def __setitem__(self, index, value):
        """
        Matrix assignment: supports M[i][j] = value and M[i,j] = value
        """
        if isinstance(index, tuple):
            if len(index) != 2:
                raise IndexError("Invalid index! Use M[i][j] or M[i, j].")
            i, j = index
            self.data[i][j] = value
        elif isinstance(index, int):
            self.data[index] = value
        else:
            raise IndexError("Invalid index type!")
    
    def __eq__(self, other):
        """
        Matrix equality check
        """
        if isinstance(other, Matrix):
            return self.n == other.n and self.m == other.m and self.data == other.data
        elif isinstance(other, list):
            return self.n == len(other) and self.m == len(other[0]) and self.data == other
        else:
            return False
    
    def assign(self, other):
        """
        Matrix assignment: set values of self to those of other
        """
        if isinstance(other, Matrix):
            if self.n != other.n or self.m != other.m:
                raise ValueError("Matrices must be the same size for assignment.")
            self.data = [row[:] for row in other.data]
        elif isinstance(other, list):
            if self.n != len(other) or self.m != len(other[0]):
                raise ValueError("Matrices must be the same size for assignment.")
            self.data = [row[:] for row in other]
        else:
            raise TypeError("Invalid type for assignment.")
    
    def shape(self):
        """
        Returns the shape of the matrix as a tuple (n, m).
        """
        return (self.n, self.m)
    
    def transpose(self):
        """
        Returns a new matrix that is the transpose of the current matrix.
        """
        transposed_data = [[self.data[j][i] for j in range(self.n)] for i in range(self.m)]
        return Matrix(data=transposed_data)
    
    def row(self, n):
        """
        Returns the nth row of the matrix as a new matrix object.
        """
        if n < 0 or n >= self.n:
            raise IndexError("Row index out of bounds.")
        return Matrix(data=[self.data[n]])
    
    def column(self, n):
        """
        Returns the nth column of the matrix as a new matrix object.
        """
        if n < 0 or n >= self.m:
            raise IndexError("Column index out of bounds.")
        return Matrix(data=[[self.data[i][n]] for i in range(self.n)])
    
    def to_list(self):
        """
        Returns the matrix as a list of lists.
        """
        return [row[:] for row in self.data]
    
    def block(self, n_0, n_1, m_0, m_1):
        """
        Returns a submatrix (block) from rows n_0 to n_1 and columns m_0 to m_1.
        """
        if n_0 < 0 or n_1 >= self.n or m_0 < 0 or m_1 >= self.m:
            raise IndexError("Block indices out of bounds.")
        block_data = [row[m_0:m_1+1] for row in self.data[n_0:n_1+1]]
        return Matrix(data=block_data)
    
    def __repr__(self):
        """
        String representation of the matrix for printing
        """
        return '\n'.join([' '.join(f"{val:4}" for val in row) for row in self.data])

In [14]:
# Test 1: Initialize a 2x3 zero matrix
M1 = Matrix(n=2, m=3)
print("M1 (2x3 zero matrix):")
print(M1)

# Test 2: Initialize a matrix from a list of lists
M2 = Matrix(data=[[1, 2, 3], [4, 5, 6]])
print("\nM2 (initialized from list of lists):")
print(M2)

# Test 3: Indexing
print("\nM2[0][1]:", M2[1][1])  # Should print 5
print("M2[1, 2]:", M2[1, 2])    # Should print 6

# Test 4: Assignment
M2[0, 2] = 45
print("\nM2 after M2[1, 2] = 45:")
print(M2)

# Test 5: Matrix equality
M3 = Matrix(data=[[1, 2, 45], [4, 5, 6]])
print("\nM2 == M3:", M2 == M3)  # Should print True

# Test 6: Matrix assignment
M4 = Matrix(n=2, m=3)
M4.assign(M2)  # Use the public method, not the dunder method
print("\nTest 6:\nTest 5:\nM4 after assignment from M2:")
print(M4)

M1 (2x3 zero matrix):
   0    0    0
   0    0    0

M2 (initialized from list of lists):
   1    2    3
   4    5    6

M2[0][1]: 5
M2[1, 2]: 6

M2 after M2[1, 2] = 45:
   1    2   45
   4    5    6

M2 == M3: True

Test 6:
Test 5:
M4 after assignment from M2:
   1    2   45
   4    5    6


In [15]:
# Test 7: Shape
print("Shape of M2:", M2.shape())  # output (2, 3)

Shape of M2: (2, 3)


In [16]:
# Test 8: Transpose
M5 = M2.transpose()
print("Transpose of M2:")
print(M5)

Transpose of M2:
   1    4
   2    5
  45    6


In [17]:
# Test 9: Row and Column
print("Row 1 of M2:")
print(M2.row(1))
print("Column 2 of M2:")
print(M2.column(2))

Row 1 of M2:
   4    5    6
Column 2 of M2:
  45
   6


In [18]:
# Test 10: To List
print("M2 as a list of lists:")
print(M2.to_list())

M2 as a list of lists:
[[1, 2, 45], [4, 5, 6]]


In [19]:
# Test 11: Block
print("Block of M2 from rows 0 to 1 and columns 1 to 2:")
print(M2.block(0, 1, 1, 2))

Block of M2 from rows 0 to 1 and columns 1 to 2:
   2   45
   5    6


In [20]:
# Test 12: Slicing 
print("Sliced M2 (rows 0 to 1, columns 1 to 2):")
print(M2[0:2, 1:3])

Sliced M2 (rows 0 to 1, columns 1 to 2):
   2   45
   5    6


3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

In [21]:
def constant(n, m, c):
    """
    Returns an n x m matrix filled with floats of value c.

    Args:
        n (int): Number of rows.
        m (int): Number of columns.
        c (float): Constant value to fill the matrix.

    Returns:
        Matrix: An n x m matrix filled with value c.

    Raises:
        ValueError: If n or m are not positive integers!
    """
    if n <= 0 or m <= 0:
        raise ValueError("Dimensions n and m must be positive integers!")
    c = float(c)  # Ensure c is a float
    return Matrix(data=[[c for _ in range(m)] for _ in range(n)])

def zeros(n, m):
    """
    Returns an n x m matrix filled with floats of value 0.
    """
    return constant(n, m, 0)

def ones(n, m):
    """
    Returns an n x m matrix filled with floats of value 1.
    """
    return constant(n, m, 1)

def eye(n):
    """
    Returns the n x n identity matrix.
    """
    if n <= 0:
        raise ValueError("Dimension n must be a positive integer!")
    identity_data = [[1.0 if i == j else 0.0 for j in range(n)] for i in range(n)]
    return Matrix(data=identity_data)

In [22]:
# Test functions
print("Constant Matrix (1x3, c=7.0):")
print(constant(1, 3, 7.0))

Constant Matrix (1x3, c=7.0):
 7.0  7.0  7.0


In [23]:
print("Zeros Matrix 2x5):")
print(zeros(2, 5))

Zeros Matrix 2x5):
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0


In [24]:
print("Ones Matrix (2x2):")
print(ones(2, 2))

Ones Matrix (2x2):
 1.0  1.0
 1.0  1.0


In [25]:
print("Identity Matrix (5x5):")
print(eye(5))

Identity Matrix (5x5):
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0


4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

In [46]:
class Matrix:
    def __init__(self, n=None, m=None, data=None):
        """
        Initialize a matrix in one of two ways:
        1. With arguments n and m, the size of the matrix (filled with zeros)
        2. With a list of lists of values
        """
        if data is not None:
            self.data = data
            self.n = len(self.data)
            if self.n == 0:
                self.m = 0
            else:
                self.m = len(self.data[0])
                if not all(len(row) == self.m for row in self.data):
                    raise ValueError("All rows must have the same number of columns.")
        elif n is not None and m is not None:
            if n <= 0 or m <= 0:
                raise ValueError("Dimensions n and m must be positive integers.")
            self.n = n
            self.m = m
            self.data = [[0 for _ in range(m)] for _ in range(n)]
        else:
            raise TypeError("Either (n and m) or data must be provided for Matrix initialization.")
    
    def scalarmul(self, c):
        """ Returns a new matrix that is the scalar product cM. """
        c = float(c)  # Ensure c is a float
        return Matrix(data=[[c * val for val in row] for row in self.data])

    def add(self, N):
        """ Adds two matrices M and N. """
        if not isinstance(N, Matrix):
            raise TypeError("Can only add a Matrix to another Matrix.")
        if self.n != N.n or self.m != N.m:
            raise ValueError("Matrices must have the same dimensions for addition.")
        return Matrix(data=[[self.data[i][j] + N.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def sub(self, N):
        """ Subtracts two matrices M and N. """
        if not isinstance(N, Matrix):
            raise TypeError("Can only subtract a Matrix from another Matrix.")
        if self.n != N.n or self.m != N.m:
            raise ValueError("Matrices must have the same dimensions for subtraction.")
        return Matrix(data=[[self.data[i][j] - N.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def mat_mult(self, N):
        """ Returns a matrix that is the matrix product of M and N. """
        if not isinstance(N, Matrix):
            raise TypeError("Can only multiply a Matrix with another Matrix.")
        if self.m != N.n:
            raise ValueError("Number of columns in M must equal number of rows in N for matrix multiplication.")
        return Matrix(data=[[sum(self.data[i][k] * N.data[k][j] for k in range(self.m)) for j in range(N.m)] for i in range(self.n)])

    def element_mult(self, N):
        """ Returns a matrix that is the element-wise product of M and N. """
        if not isinstance(N, Matrix):
            raise TypeError("Can only perform element-wise multiplication with another Matrix.")
        if self.n != N.n or self.m != N.m:
            raise ValueError("Matrices must have the same dimensions for element-wise multiplication.")
        return Matrix(data=[[self.data[i][j] * N.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def equals(self, N):
        """ Returns True if M == N, False otherwise. """
        if not isinstance(N, Matrix):
            return False
        return self.n == N.n and self.m == N.m and self.data == N.data

    def __repr__(self):
        """ String representation of the matrix for printing. """
        return '\n'.join([' '.join(f"{val:4}" for val in row) for row in self.data])

In [27]:
# Testing the new functions
M1 = Matrix(data=[[1, 2], [3, 4]])
M2 = Matrix(data=[[5, 6], [7, 8]])

print("Scalar multiplication (M1 * 2):")
print(M1.scalarmul(2))

Scalar multiplication (M1 * 2):
 2.0  4.0
 6.0  8.0


In [28]:
print("Matrix addition (M1 + M2):")
print(M1.add(M2))

Matrix addition (M1 + M2):
   6    8
  10   12


In [29]:
print("Matrix subtraction (M1 - M2):")
print(M1.sub(M2))

Matrix subtraction (M1 - M2):
  -4   -4
  -4   -4


In [30]:
print("Matrix multiplication (M1 * M2):")
print(M1.mat_mult(M2))

Matrix multiplication (M1 * M2):
  19   22
  43   50


In [31]:
print("Element-wise multiplication (M1 * M2):")
print(M1.element_mult(M2))

Element-wise multiplication (M1 * M2):
   5   12
  21   32


5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


In [95]:
class Matrix:
    def __init__(self, n=None, m=None, data=None):
        if data is not None:
            self.data = data
            self.n = len(self.data)
            if self.n == 0:
                self.m = 0
            else:
                self.m = len(self.data[0])
                if not all(len(row) == self.m for row in self.data):
                    raise ValueError("All rows must have the same number of columns!")
        elif n is not None and m is not None:
            if n <= 0 or m <= 0:
                raise ValueError("Dimensions n and m must be positive integers!")
            self.n = n
            self.m = m
            self.data = [[0 for _ in range(m)] for _ in range(n)]
        else:
            raise TypeError("Either (n and m) or data must be provided for Matrix initialization.")
    
    def scalarmul(self, c):
        """ Returns a new matrix that is the scalar product cM. """
        c = float(c)  # Ensure c is a float
        return Matrix(data=[[c * val for val in row] for row in self.data])

    def add(self, N):
        """ Adds two matrices M and N. """
        if not isinstance(N, Matrix) or self.n != N.n or self.m != N.m:
            raise ValueError("Matrices must have the same dimensions for addition.")
        return Matrix(data=[[self.data[i][j] + N.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def sub(self, N):
        """ Subtracts two matrices M and N. """
        if not isinstance(N, Matrix) or self.n != N.n or self.m != N.m:
            raise ValueError("Matrices must have the same dimensions for subtraction.")
        return Matrix(data=[[self.data[i][j] - N.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def mat_mult(self, N):
        """ Returns a matrix that is the matrix product of M and N. """
        if not isinstance(N, Matrix) or self.m != N.n:
            raise ValueError("Number of columns in M must equal number of rows in N for matrix multiplication.")
        return Matrix(data=[[sum(self.data[i][k] * N.data[k][j] for k in range(self.m)) for j in range(N.m)] for i in range(self.n)])

    def element_mult(self, N):
        """ Returns a matrix that is the element-wise product of M and N. """
        if not isinstance(N, Matrix) or self.n != N.n or self.m != N.m:
            raise ValueError("Matrices must have the same dimensions for element-wise multiplication.")
        return Matrix(data=[[self.data[i][j] * N.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def equals(self, N):
        """ Returns True if M == N, False otherwise """
        return isinstance(N, Matrix) and self.n == N.n and self.m == N.m and self.data == N.data

    def __repr__(self):
        """ Improved string representation of the matrix for cleaner printing """
        return '\n'.join([' '.join(map(str, row)) for row in self.data])

    # Operator Overloading
    def __mul__(self, other):
        """
        Overloads the * operator for:
        - Scalar multiplication (M * c or c * M)
        - Matrix multiplication (M * N)
        - Element-wise multiplication (if matrices have the same size)
        """
        if isinstance(other, (int, float)):
            return self.scalarmul(other)
        elif isinstance(other, Matrix):
            if self.n == other.n and self.m == other.m:
                return self.element_mult(other)  # Element-wise multiplication
            else:
                return self.mat_mult(other)  # Matrix multiplication
        else:
            raise TypeError("Unsupported operand type for *.")

    def __rmul__(self, other):
        """
        Overloads the * operator for scalar multiplication when the scalar is on the left (c * M).
        """
        if isinstance(other, (int, float)):
            return self.scalarmul(other)
        else:
            raise TypeError("Unsupported operand type for *.")

    def __add__(self, other):
        """
        Overloads the + operator for matrix addition (M + N).
        """
        if isinstance(other, Matrix):
            return self.add(other)
        else:
            raise TypeError("Unsupported operand type for +.")

    def __sub__(self, other):
        """
        Overloads the - operator for matrix subtraction (M - N).
        """
        if isinstance(other, Matrix):
            return self.sub(other)
        else:
            raise TypeError("Unsupported operand type for -.")

    def __eq__(self, other):
        """
        Overloads the == operator for matrix equality (M == N).
        """
        return self.equals(other)

    def __ne__(self, other):
        """
        Overloads the != operator for matrix inequality (M != N).
        """
        return not self.__eq__(other)

In [96]:
# Testing Matrix class
M1 = Matrix(data=[[1, 2], [3, 4]])
M2 = Matrix(data=[[5, 6], [7, 8]])

# Scalar multiplication
print("Scalar multiplication (2 * M1):")
print(2 * M1)

print("\nScalar multiplication (M1 * 2):")
print(M1 * 2)

Scalar multiplication (2 * M1):
2.0 4.0
6.0 8.0

Scalar multiplication (M1 * 2):
2.0 4.0
6.0 8.0


In [34]:
# Matrix addition
print("Matrix addition (M1 + M2):")
print(M1 + M2)

Matrix addition (M1 + M2):
6 8
10 12


In [35]:
# Matrix subtraction
print("Matrix subtraction (M1 - M2):")
print(M1 - M2)

Matrix subtraction (M1 - M2):
-4 -4
-4 -4


In [36]:
# Matrix multiplication
print("Matrix multiplication (M1 * M2):")
print(M1 * M2)

Matrix multiplication (M1 * M2):
5 12
21 32


In [37]:
# Element-wise multiplication (Hadamard product)
print("Element-wise multiplication (M1 * M2):")
print(M1 * M2)

Element-wise multiplication (M1 * M2):
5 12
21 32


In [38]:
# Matrix equality
print("Matrix equality (M1 == M2):", M1 == M2)
print("Matrix equality (M1 == M1):", M1 == M1)

Matrix equality (M1 == M2): False
Matrix equality (M1 == M1): True


In [39]:
# Matrix inequality
print("Matrix inequality (M1 != M2):", M1 != M2)
print("Matrix inequality (M1 != M1):", M1 != M1)

Matrix inequality (M1 != M2): True
Matrix inequality (M1 != M1): False


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [99]:

def __mul__(self, other):
    """
    Matrix multiplication: A * B
    """
    if not isinstance(other, Matrix):
        raise TypeError("Matrix multiplication requires another Matrix")
    
    if self.m != other.n:
        raise ValueError(f"Matrix dimensions don't match for multiplication: ({self.n},{self.m}) and ({other.n},{other.m})")
    
    # Initialize the result matrix
    result = [[0.0 for _ in range(other.m)] for _ in range(self.n)]
    
    # perform matrix multiplication
    for i in range(self.n):
        for j in range(other.m):
            for k in range(self.m):
                result[i][j] += self.data[i][k] * other.data[k][j]
    
    return Matrix(data=result)

def __add__(self, other):
    """
    Matrix addition: A + B
    """
    if not isinstance(other, Matrix):
        raise TypeError("Matrix addition requires another Matrix")
    
    if self.n != other.n or self.m != other.m:
        raise ValueError(f"Matrix dimensions don't match for addition: ({self.n},{self.m}) and ({other.n},{other.m})")
    
    # Initialize the result matrix
    result = [[self.data[i][j] + other.data[i][j] for j in range(self.m)] for i in range(self.n)]
    
    return Matrix(data=result)

In [100]:
# Create example matrices A, B, and C
A = Matrix(data=[[1, 2], [3, 4]])
B = Matrix(data=[[5, 6], [7, 8]])
C = Matrix(data=[[9, 10], [11, 12]])

# Identity matrix
I = eye(2)  # Using previously defined eye function

print("Matrix A:")
print(A)
print("\nMatrix B:")
print(B)
print("\nMatrix C:")
print(C)
print("\nIdentity Matrix I:")
print(I)

Matrix A:
1 2
3 4

Matrix B:
5 6
7 8

Matrix C:
9 10
11 12

Identity Matrix I:
1.0 0.0
0.0 1.0


In [101]:
# Property 1: Associativity of Matrix Multiplication (AB)C = A(BC)
AB = A * B
AB_C = AB * C
BC = B * C
A_BC = A * BC

print("\nProperty 1: Associativity of Matrix Multiplication")
print("(AB)C:")
print(AB_C)
print("\nA(BC):")
print(A_BC)
print("\nAre (AB)C and A(BC) equal?", AB_C == A_BC)


Property 1: Associativity of Matrix Multiplication
(AB)C:
45 120
231 384

A(BC):
45 120
231 384

Are (AB)C and A(BC) equal? True


In [102]:
# Property 2: Distributivity of Matrix Multiplication over Addition A(B + C) = AB + AC
B_plus_C = B + C
A_B_plus_C = A * B_plus_C
AB = A * B
AC = A * C
AB_plus_AC = AB + AC

print("Property 2: Distributivity of Matrix Multiplication over Addition")
print("A(B + C):")
print(A_B_plus_C)
print("\nAB + AC:")
print(AB_plus_AC)
print("\nAre A(B + C) and AB + AC equal?", A_B_plus_C == AB_plus_AC)

Property 2: Distributivity of Matrix Multiplication over Addition
A(B + C):
14 32
54 80

AB + AC:
14 32
54 80

Are A(B + C) and AB + AC equal? True


In [103]:
# Property 3: Non-Commutativity of Matrix Multiplication AB ≠ BA
AB = A * B
BA = B * A

print("Property 3: Non-Commutativity of Matrix Multiplication")
print("AB:")
print(AB)
print("\nBA:")
print(BA)
print("\nAre AB and BA equal?", AB == BA)

Property 3: Non-Commutativity of Matrix Multiplication
AB:
5 12
21 32

BA:
5 12
21 32

Are AB and BA equal? True


In [105]:
# Property 4: Identity Matrix Property AI = A
AI = A * I

print("Property 4: Identity Matrix Property")
print("AI:")
print(AI)
print("\nA:")
print(A)
print("\nAre AI and A equal?", AI == A)

Property 4: Identity Matrix Property
AI:
1.0 0.0
0.0 4.0

A:
1 2
3 4

Are AI and A equal? False
